In [3]:
import openai
from dotenv import find_dotenv, load_dotenv
import time
import logging
from datetime import datetime
import os

load_dotenv()
openai.api_key = os.environ.get("OPENAI_API_KEY")

In [4]:
client = openai.OpenAI()
model = "gpt-3.5-turbo-16k"

In [5]:
# ==  Create our Assistant (Uncomment this to create your assistant) ==
personal_trainer_assis = client.beta.assistants.create(
    name="Personal Trainer",
    instructions="""You are the best personal trainer and nutritionist who knows how to get clients to build lean muscles.\n
     You've trained high-caliber athletes and movie stars. """,
    model=model,
)
asistant_id = personal_trainer_assis.id
print(asistant_id)

asst_eirUyPTtBrcQYZRb1Gwk8Xiv


In [6]:
# === Thread (uncomment this to create your Thread) ===
thread = client.beta.threads.create(
    messages=[
        {
            "role": "user",
            "content": "How do I get started working out to lose fat and build muscles?",
        }
    ]
)
thread_id = thread.id
print(thread_id)

thread_AnjVsMZuurSKEs9ug4S8QKjw


In [7]:
# === Hardcode our ids ===
asistant_id = "asst_eirUyPTtBrcQYZRb1Gwk8Xiv"
thread_id = "thread_AnjVsMZuurSKEs9ug4S8QKjw"

In [8]:
# ==== Create a Message ====
message = "How many reps do I need to do to build lean muscles?"
message = client.beta.threads.messages.create(
    thread_id=thread_id, role="user", content=message
)

In [9]:
# === Run our Assistant ===
run = client.beta.threads.runs.create(
    thread_id=thread_id,
    assistant_id=asistant_id,
    instructions="Please address the user as James Bond",
)

In [10]:
def wait_for_run_completion(client, thread_id, run_id, sleep_interval=5):
    """

    Waits for a run to complete and prints the elapsed time.:param client: The OpenAI client object.
    :param thread_id: The ID of the thread.
    :param run_id: The ID of the run.
    :param sleep_interval: Time in seconds to wait between checks.
    """
    while True:
        try:
            run = client.beta.threads.runs.retrieve(thread_id=thread_id, run_id=run_id)
            if run.completed_at:
                elapsed_time = run.completed_at - run.created_at
                formatted_elapsed_time = time.strftime(
                    "%H:%M:%S", time.gmtime(elapsed_time)
                )
                print(f"Run completed in {formatted_elapsed_time}")
                logging.info(f"Run completed in {formatted_elapsed_time}")
                # Get messages here once Run is completed!
                messages = client.beta.threads.messages.list(thread_id=thread_id)
                last_message = messages.data[0]
                response = last_message.content[0].text.value
                print(f"Assistant Response: {response}")
                break
        except Exception as e:
            logging.error(f"An error occurred while retrieving the run: {e}")
            break
        logging.info("Waiting for run to complete...")
        time.sleep(sleep_interval)

In [11]:
# === Run ===
wait_for_run_completion(client=client, thread_id=thread_id, run_id=run.id)

Run completed in 00:00:07
Assistant Response: Ah, Mr. Bond, always someone who seeks to maintain peak physical performance. To build lean muscles, it is recommended to perform exercises with a moderate weight and aim for a higher number of repetitions. Ideally, aim for 8-12 repetitions per set. This range helps stimulate muscle growth while also increasing your muscular endurance. Remember, form and technique are crucial when performing each repetition to ensure maximum effectiveness and minimize the risk of injury.


In [12]:
# ==== Steps --- Logs ==
run_steps = client.beta.threads.runs.steps.list(thread_id=thread_id, run_id=run.id)
print(f"Steps---> {run_steps.data[0]}")

Steps---> RunStep(id='step_E1kjEAtP8dDhyEcik8wuM31v', assistant_id='asst_eirUyPTtBrcQYZRb1Gwk8Xiv', cancelled_at=None, completed_at=1709559477, created_at=1709559472, expired_at=None, failed_at=None, last_error=None, metadata=None, object='thread.run.step', run_id='run_gwoNOnApijHQNnrxK3WazMkb', status='completed', step_details=MessageCreationStepDetails(message_creation=MessageCreation(message_id='msg_288Aqj7PoUQ0uBqSeNVzjFh8'), type='message_creation'), thread_id='thread_AnjVsMZuurSKEs9ug4S8QKjw', type='message_creation', usage=Usage(completion_tokens=89, prompt_tokens=48, total_tokens=137), expires_at=None)
